In [2]:
import pandas as pd
import numpy as np
import math
from scipy import signal

import tqdm
from tqdm.notebook import tqdm
import time

import pingouin as pg

import statistics

#現在の最大表示列数の出力
pd.get_option("display.max_columns")

#最大表示列数の指定（ここでは50列を指定）
pd.set_option('display.max_columns', 50)

pd.get_option("display.max_rows")
pd.set_option("display.max_rows",100)

# データの読み込み

In [6]:
df_ori_f_columns = ['user', 'doc', 'stroke_inter', 'stroke_duration', 'start_x', 'start_y', 'stop_x', 'stop_y',
                            'direct_ete_distance', 'mean_result_leng', 'flag', 'direct_ete_line', 'phone',
                            '20_pairwise_v', '50_pairwise_v', '80_pairwise_v', '20_pairwise_acc', '50_pairwise_acc',
                            '80_pairwise_acc', '3ots_m_v', 'ete_larg_deviation', '20_ete_line', '50_ete_line',
                            '80_ete_line', 'ave_direction', 'length_trajectory', 'ratio_ete', 'ave_v', '5points_m_acc',
                            'm_stroke_press', 'm_stroke_area_cover', 'finger_orien', 'cd_finger_orien', 'phone_orien']
        

In [5]:
# read featMat.csv
df_fm = pd.read_csv('../../02_features/featMat.csv')
df_fm.head()

,37,4,0.341,0.256,27.416,-27.113,12.398,-32.96,16.116,0.35736,4.1,-2.7704,1,0.41784,4.1784,2719.2,-6.0418e+005,-432.24,6.0332e+005,4.1784.1,0.17777,0,0.1,0.12444,-2.5812,16.132,0.99898,63.018,-432.24.1,0.32,0.044444,0.2,0.3,1.1
0,37,4,0.913,0.086,21.771,-30.037,9.8778,-33.867,12.4950,0.26504,4,-2.830000,1,0.000,156.790,502.37,-26083.0,5117.7,116600.0,265.140,0.78867,0.000000,0.00000,0.406210,-2.199400,12.5960,0.99197,146.470,5117.7,0.600,0.088889,0.0,0,1
1,37,4,0.733,0.111,10.886,-34.169,26.9120,-34.673,16.0340,0.99118,2,-0.031436,1,32.122,143.850,263.58,-52140.0,-3563.5,26382.0,188.820,-1.06510,-0.802530,-0.11658,0.000000,0.086288,16.2030,0.98959,145.970,-3563.5,0.480,0.044444,0.0,0,1
2,37,4,0.448,0.094,19.151,-34.572,12.7000,-36.790,6.8213,0.59647,4,-2.810500,1,17.360,76.638,230.62,-10012.0,-4604.1,25874.0,95.857,0.60170,0.000000,0.10128,0.570430,-2.723100,6.9674,0.97902,74.122,-4604.1,0.400,0.044444,0.0,0,1
3,37,4,0.519,0.056,27.718,-32.758,14.6150,-37.294,13.8660,0.67819,4,-2.808300,1,74.821,275.570,993.21,-126180.0,-21377.0,217040.0,244.680,-0.10990,-0.096714,0.00000,0.001099,-2.717000,13.8710,0.99965,247.690,-21377.0,0.440,0.044444,0.0,0,1
4,37,4,0.290,0.115,30.137,-30.641,15.8250,-35.479,15.1080,0.79869,4,-2.815600,1,77.591,132.640,226.95,-10820.0,-2632.8,16290.0,122.930,-0.57426,-0.434660,0.00000,0.092527,-2.791400,15.2220,0.99254,132.360,2831.1,0.405,0.044444,0.0,0,1


In [12]:
df_fm2 = pd.read_csv('../../02_features/featMat.csv', names=df_ori_f_columns)
df_fm2.head()

,user,doc,stroke_inter,stroke_duration,start_x,start_y,stop_x,stop_y,direct_ete_distance,mean_result_leng,flag,direct_ete_line,phone,20_pairwise_v,50_pairwise_v,80_pairwise_v,20_pairwise_acc,50_pairwise_acc,80_pairwise_acc,3ots_m_v,ete_larg_deviation,20_ete_line,50_ete_line,80_ete_line,ave_direction,length_trajectory,ratio_ete,ave_v,5points_m_acc,m_stroke_press,m_stroke_area_cover,finger_orien,cd_finger_orien,phone_orien
0,37,4,0.341,0.256,27.416,-27.113,12.3980,-32.960,16.1160,0.35736,4,-2.770400,1,0.41784,4.1784,2719.20,-604180.0,-432.24,603320.0,4.1784,0.17777,0.000000,0.00000,0.124440,-2.581200,16.1320,0.99898,63.018,-432.24,0.32,0.044444,0.0,0,1
1,37,4,0.913,0.086,21.771,-30.037,9.8778,-33.867,12.4950,0.26504,4,-2.830000,1,0.00000,156.7900,502.37,-26083.0,5117.70,116600.0,265.1400,0.78867,0.000000,0.00000,0.406210,-2.199400,12.5960,0.99197,146.470,5117.70,0.60,0.088889,0.0,0,1
2,37,4,0.733,0.111,10.886,-34.169,26.9120,-34.673,16.0340,0.99118,2,-0.031436,1,32.12200,143.8500,263.58,-52140.0,-3563.50,26382.0,188.8200,-1.06510,-0.802530,-0.11658,0.000000,0.086288,16.2030,0.98959,145.970,-3563.50,0.48,0.044444,0.0,0,1
3,37,4,0.448,0.094,19.151,-34.572,12.7000,-36.790,6.8213,0.59647,4,-2.810500,1,17.36000,76.6380,230.62,-10012.0,-4604.10,25874.0,95.8570,0.60170,0.000000,0.10128,0.570430,-2.723100,6.9674,0.97902,74.122,-4604.10,0.40,0.044444,0.0,0,1
4,37,4,0.519,0.056,27.718,-32.758,14.6150,-37.294,13.8660,0.67819,4,-2.808300,1,74.82100,275.5700,993.21,-126180.0,-21377.00,217040.0,244.6800,-0.10990,-0.096714,0.00000,0.001099,-2.717000,13.8710,0.99965,247.690,-21377.00,0.44,0.044444,0.0,0,1


In [4]:
# read frank_features_calc.csv

df_ffc = pd.read_csv('frank_features_calc.csv')
df_ffc.head()

,37.0,4.0,0.3410000801086426,0.2560000419616699,27.415873015873014,-27.11349206349206,12.397619047619047,-32.9595238095238,16.115956050777346,0.35736492930930625,4.0.1,-2.77037753059156,1.0,2.0892222953311066,4.178444590662213,1512.5140134152455,-432.23945636866847,-432.23945636866847.1,-432.23945636866847.2,1512.5140134152455.1,9.172222222222226,0.4031746031746053,4.989285714285718,9.172222222222226.1,-2.581235073441599,16.132491204484552,0.9989750402775605,63.01753343813912,-432.23945636866847.3,0.32,0.04444445,0.0,0.0.1,1.0.1
0,37.0,4.0,0.913,0.086,21.771429,-30.036508,9.877778,-33.866667,12.495161,0.265040,4.0,-2.830046,1.0,0.000000,156.788711,502.370053,-21322.709457,5117.745334,88903.936269,394.015610,8.063492,0.000000,6.350000,8.063492,-2.199436,12.596318,0.991969,146.468877,5117.745334,0.600,0.088889,0.0,0.0,1.0
1,37.0,4.0,0.733,0.111,10.885714,-34.169048,26.911905,-34.673016,16.034113,0.991178,2.0,-0.031436,1.0,32.122213,143.853750,263.584103,-38945.522228,-3563.474273,18510.405771,188.821551,15.522222,-15.522222,-4.636508,-1.209524,0.086288,16.202833,0.989587,145.971705,-3563.474273,0.480,0.044444,0.0,0.0,1.0
2,37.0,4.0,0.448,0.094,19.150794,-34.572222,12.700000,-36.789683,6.821281,0.596472,4.0,-2.810497,1.0,17.359942,76.637825,230.616519,-9135.221517,-4604.073096,17834.521764,95.856637,4.233333,1.007937,3.225397,4.233333,-2.723130,6.967444,0.979022,74.121670,-4604.073096,0.400,0.044444,0.0,0.0,1.0
3,37.0,4.0,0.519,0.056,27.718254,-32.757937,14.615079,-37.293651,13.865998,0.678191,4.0,-2.808348,1.0,106.887510,275.572786,767.677216,-126175.290213,-21377.012627,217042.986149,244.681884,8.567460,2.066270,6.249206,8.013095,-2.717012,13.870901,0.999646,247.694691,-21377.012627,0.440,0.044444,0.0,0.0,1.0
4,37.0,4.0,0.290,0.115,30.137302,-30.641270,15.824603,-35.479365,15.108292,0.798693,4.0,-2.815623,1.0,77.591293,132.640410,226.948167,-10564.856388,-2632.815901,15764.657822,180.735884,9.474603,1.108730,3.930952,8.769048,-2.791383,15.221881,0.992538,132.364169,2831.120670,0.405,0.044444,0.0,0.0,1.0


In [11]:
df_ffc2 = pd.read_csv('frank_features_calc.csv', names=df_ori_f_columns)
df_ffc2.head()

,user,doc,stroke_inter,stroke_duration,start_x,start_y,stop_x,stop_y,direct_ete_distance,mean_result_leng,flag,direct_ete_line,phone,20_pairwise_v,50_pairwise_v,80_pairwise_v,20_pairwise_acc,50_pairwise_acc,80_pairwise_acc,3ots_m_v,ete_larg_deviation,20_ete_line,50_ete_line,80_ete_line,ave_direction,length_trajectory,ratio_ete,ave_v,5points_m_acc,m_stroke_press,m_stroke_area_cover,finger_orien,cd_finger_orien,phone_orien
0,37.0,4.0,0.341,0.256,27.415873,-27.113492,12.397619,-32.959524,16.115956,0.357365,4.0,-2.770378,1.0,2.089222,4.178445,1512.514013,-432.239456,-432.239456,-432.239456,1512.514013,9.172222,0.403175,4.989286,9.172222,-2.581235,16.132491,0.998975,63.017533,-432.239456,0.32,0.044444,0.0,0.0,1.0
1,37.0,4.0,0.913,0.086,21.771429,-30.036508,9.877778,-33.866667,12.495161,0.265040,4.0,-2.830046,1.0,0.000000,156.788711,502.370053,-21322.709457,5117.745334,88903.936269,394.015610,8.063492,0.000000,6.350000,8.063492,-2.199436,12.596318,0.991969,146.468877,5117.745334,0.60,0.088889,0.0,0.0,1.0
2,37.0,4.0,0.733,0.111,10.885714,-34.169048,26.911905,-34.673016,16.034113,0.991178,2.0,-0.031436,1.0,32.122213,143.853750,263.584103,-38945.522228,-3563.474273,18510.405771,188.821551,15.522222,-15.522222,-4.636508,-1.209524,0.086288,16.202833,0.989587,145.971705,-3563.474273,0.48,0.044444,0.0,0.0,1.0
3,37.0,4.0,0.448,0.094,19.150794,-34.572222,12.700000,-36.789683,6.821281,0.596472,4.0,-2.810497,1.0,17.359942,76.637825,230.616519,-9135.221517,-4604.073096,17834.521764,95.856637,4.233333,1.007937,3.225397,4.233333,-2.723130,6.967444,0.979022,74.121670,-4604.073096,0.40,0.044444,0.0,0.0,1.0
4,37.0,4.0,0.519,0.056,27.718254,-32.757937,14.615079,-37.293651,13.865998,0.678191,4.0,-2.808348,1.0,106.887510,275.572786,767.677216,-126175.290213,-21377.012627,217042.986149,244.681884,8.567460,2.066270,6.249206,8.013095,-2.717012,13.870901,0.999646,247.694691,-21377.012627,0.44,0.044444,0.0,0.0,1.0


In [10]:
# この方法だと一番最初の行が消滅する
df_ffc.columns = df_ori_f_columns
df_ffc.head()

,user,doc,stroke_inter,stroke_duration,start_x,start_y,stop_x,stop_y,direct_ete_distance,mean_result_leng,flag,direct_ete_line,phone,20_pairwise_v,50_pairwise_v,80_pairwise_v,20_pairwise_acc,50_pairwise_acc,80_pairwise_acc,3ots_m_v,ete_larg_deviation,20_ete_line,50_ete_line,80_ete_line,ave_direction,length_trajectory,ratio_ete,ave_v,5points_m_acc,m_stroke_press,m_stroke_area_cover,finger_orien,cd_finger_orien,phone_orien
0,37.0,4.0,0.913,0.086,21.771429,-30.036508,9.877778,-33.866667,12.495161,0.265040,4.0,-2.830046,1.0,0.000000,156.788711,502.370053,-21322.709457,5117.745334,88903.936269,394.015610,8.063492,0.000000,6.350000,8.063492,-2.199436,12.596318,0.991969,146.468877,5117.745334,0.600,0.088889,0.0,0.0,1.0
1,37.0,4.0,0.733,0.111,10.885714,-34.169048,26.911905,-34.673016,16.034113,0.991178,2.0,-0.031436,1.0,32.122213,143.853750,263.584103,-38945.522228,-3563.474273,18510.405771,188.821551,15.522222,-15.522222,-4.636508,-1.209524,0.086288,16.202833,0.989587,145.971705,-3563.474273,0.480,0.044444,0.0,0.0,1.0
2,37.0,4.0,0.448,0.094,19.150794,-34.572222,12.700000,-36.789683,6.821281,0.596472,4.0,-2.810497,1.0,17.359942,76.637825,230.616519,-9135.221517,-4604.073096,17834.521764,95.856637,4.233333,1.007937,3.225397,4.233333,-2.723130,6.967444,0.979022,74.121670,-4604.073096,0.400,0.044444,0.0,0.0,1.0
3,37.0,4.0,0.519,0.056,27.718254,-32.757937,14.615079,-37.293651,13.865998,0.678191,4.0,-2.808348,1.0,106.887510,275.572786,767.677216,-126175.290213,-21377.012627,217042.986149,244.681884,8.567460,2.066270,6.249206,8.013095,-2.717012,13.870901,0.999646,247.694691,-21377.012627,0.440,0.044444,0.0,0.0,1.0
4,37.0,4.0,0.290,0.115,30.137302,-30.641270,15.824603,-35.479365,15.108292,0.798693,4.0,-2.815623,1.0,77.591293,132.640410,226.948167,-10564.856388,-2632.815901,15764.657822,180.735884,9.474603,1.108730,3.930952,8.769048,-2.791383,15.221881,0.992538,132.364169,2831.120670,0.405,0.044444,0.0,0.0,1.0


# 実際に実験で使用するデータの中身(欠損値処理などを施す)

# featMat.csv

In [20]:
# 欠損値のカウント: Trueは欠損値
df_fm2.isnull().apply(lambda col: col.value_counts(), axis=0).fillna(0).astype(np.int)

,user,doc,stroke_inter,stroke_duration,start_x,start_y,stop_x,stop_y,direct_ete_distance,mean_result_leng,flag,direct_ete_line,phone,20_pairwise_v,50_pairwise_v,80_pairwise_v,20_pairwise_acc,50_pairwise_acc,80_pairwise_acc,3ots_m_v,ete_larg_deviation,20_ete_line,50_ete_line,80_ete_line,ave_direction,length_trajectory,ratio_ete,ave_v,5points_m_acc,m_stroke_press,m_stroke_area_cover,finger_orien,cd_finger_orien,phone_orien
False,21158,21158,21119,21158,21158,21158,21158,21158,21158,21158,21158,21158,21158,21158,21158,21158,20895,20899,20902,21157,21158,21158,21158,21158,21158,21158,20896,21158,20901,21158,21158,21158,21158,21158
True,0,0,39,0,0,0,0,0,0,0,0,0,0,0,0,0,263,259,256,1,0,0,0,0,0,0,262,0,257,0,0,0,0,0


In [13]:
# featMat
df_fm2_drop = df_fm2.drop({'phone', 'finger_orien', 'cd_finger_orien', 'phone_orien'}, axis=1)
df_fm2_drop.head()

,user,doc,stroke_inter,stroke_duration,start_x,start_y,stop_x,stop_y,direct_ete_distance,mean_result_leng,flag,direct_ete_line,20_pairwise_v,50_pairwise_v,80_pairwise_v,20_pairwise_acc,50_pairwise_acc,80_pairwise_acc,3ots_m_v,ete_larg_deviation,20_ete_line,50_ete_line,80_ete_line,ave_direction,length_trajectory,ratio_ete,ave_v,5points_m_acc,m_stroke_press,m_stroke_area_cover
0,37,4,0.341,0.256,27.416,-27.113,12.3980,-32.960,16.1160,0.35736,4,-2.770400,0.41784,4.1784,2719.20,-604180.0,-432.24,603320.0,4.1784,0.17777,0.000000,0.00000,0.124440,-2.581200,16.1320,0.99898,63.018,-432.24,0.32,0.044444
1,37,4,0.913,0.086,21.771,-30.037,9.8778,-33.867,12.4950,0.26504,4,-2.830000,0.00000,156.7900,502.37,-26083.0,5117.70,116600.0,265.1400,0.78867,0.000000,0.00000,0.406210,-2.199400,12.5960,0.99197,146.470,5117.70,0.60,0.088889
2,37,4,0.733,0.111,10.886,-34.169,26.9120,-34.673,16.0340,0.99118,2,-0.031436,32.12200,143.8500,263.58,-52140.0,-3563.50,26382.0,188.8200,-1.06510,-0.802530,-0.11658,0.000000,0.086288,16.2030,0.98959,145.970,-3563.50,0.48,0.044444
3,37,4,0.448,0.094,19.151,-34.572,12.7000,-36.790,6.8213,0.59647,4,-2.810500,17.36000,76.6380,230.62,-10012.0,-4604.10,25874.0,95.8570,0.60170,0.000000,0.10128,0.570430,-2.723100,6.9674,0.97902,74.122,-4604.10,0.40,0.044444
4,37,4,0.519,0.056,27.718,-32.758,14.6150,-37.294,13.8660,0.67819,4,-2.808300,74.82100,275.5700,993.21,-126180.0,-21377.00,217040.0,244.6800,-0.10990,-0.096714,0.00000,0.001099,-2.717000,13.8710,0.99965,247.690,-21377.00,0.44,0.044444


In [15]:
df_fm2_drop.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21158 entries, 0 to 21157
Data columns (total 30 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   user                 21158 non-null  int64  
 1   doc                  21158 non-null  int64  
 2   stroke_inter         21119 non-null  float64
 3   stroke_duration      21158 non-null  float64
 4   start_x              21158 non-null  float64
 5   start_y              21158 non-null  float64
 6   stop_x               21158 non-null  float64
 7   stop_y               21158 non-null  float64
 8   direct_ete_distance  21158 non-null  float64
 9   mean_result_leng     21158 non-null  float64
 10  flag                 21158 non-null  int64  
 11  direct_ete_line      21158 non-null  float64
 12  20_pairwise_v        21158 non-null  float64
 13  50_pairwise_v        21158 non-null  float64
 14  80_pairwise_v        21158 non-null  float64
 15  20_pairwise_acc      20895 non-null 

In [19]:
# Falseの数え上げ
# df_fm2_drop.isnull().count()
# 欠損値のカウント: Trueは欠損値
df_fm2_drop.isnull().apply(lambda col: col.value_counts(), axis=0).fillna(0).astype(np.int)

,user,doc,stroke_inter,stroke_duration,start_x,start_y,stop_x,stop_y,direct_ete_distance,mean_result_leng,flag,direct_ete_line,20_pairwise_v,50_pairwise_v,80_pairwise_v,20_pairwise_acc,50_pairwise_acc,80_pairwise_acc,3ots_m_v,ete_larg_deviation,20_ete_line,50_ete_line,80_ete_line,ave_direction,length_trajectory,ratio_ete,ave_v,5points_m_acc,m_stroke_press,m_stroke_area_cover
False,21158,21158,21119,21158,21158,21158,21158,21158,21158,21158,21158,21158,21158,21158,21158,20895,20899,20902,21157,21158,21158,21158,21158,21158,21158,20896,21158,20901,21158,21158
True,0,0,39,0,0,0,0,0,0,0,0,0,0,0,0,263,259,256,1,0,0,0,0,0,0,262,0,257,0,0


In [25]:
# 欠損値削除: how='any'で欠損値が1つでも含まれる行(axis=0)を削除
df_fm2_drop_f = df_fm2_drop.dropna(axis=0, how='any')
df_fm2_drop_f.head()

,user,doc,stroke_inter,stroke_duration,start_x,start_y,stop_x,stop_y,direct_ete_distance,mean_result_leng,flag,direct_ete_line,20_pairwise_v,50_pairwise_v,80_pairwise_v,20_pairwise_acc,50_pairwise_acc,80_pairwise_acc,3ots_m_v,ete_larg_deviation,20_ete_line,50_ete_line,80_ete_line,ave_direction,length_trajectory,ratio_ete,ave_v,5points_m_acc,m_stroke_press,m_stroke_area_cover
0,37,4,0.341,0.256,27.416,-27.113,12.3980,-32.960,16.1160,0.35736,4,-2.770400,0.41784,4.1784,2719.20,-604180.0,-432.24,603320.0,4.1784,0.17777,0.000000,0.00000,0.124440,-2.581200,16.1320,0.99898,63.018,-432.24,0.32,0.044444
1,37,4,0.913,0.086,21.771,-30.037,9.8778,-33.867,12.4950,0.26504,4,-2.830000,0.00000,156.7900,502.37,-26083.0,5117.70,116600.0,265.1400,0.78867,0.000000,0.00000,0.406210,-2.199400,12.5960,0.99197,146.470,5117.70,0.60,0.088889
2,37,4,0.733,0.111,10.886,-34.169,26.9120,-34.673,16.0340,0.99118,2,-0.031436,32.12200,143.8500,263.58,-52140.0,-3563.50,26382.0,188.8200,-1.06510,-0.802530,-0.11658,0.000000,0.086288,16.2030,0.98959,145.970,-3563.50,0.48,0.044444
3,37,4,0.448,0.094,19.151,-34.572,12.7000,-36.790,6.8213,0.59647,4,-2.810500,17.36000,76.6380,230.62,-10012.0,-4604.10,25874.0,95.8570,0.60170,0.000000,0.10128,0.570430,-2.723100,6.9674,0.97902,74.122,-4604.10,0.40,0.044444
4,37,4,0.519,0.056,27.718,-32.758,14.6150,-37.294,13.8660,0.67819,4,-2.808300,74.82100,275.5700,993.21,-126180.0,-21377.00,217040.0,244.6800,-0.10990,-0.096714,0.00000,0.001099,-2.717000,13.8710,0.99965,247.690,-21377.00,0.44,0.044444


In [26]:
# 欠損値のカウント: Trueは欠損値
df_fm2_drop_f.isnull().apply(lambda col: col.value_counts(), axis=0).fillna(0).astype(np.int)

,user,doc,stroke_inter,stroke_duration,start_x,start_y,stop_x,stop_y,direct_ete_distance,mean_result_leng,flag,direct_ete_line,20_pairwise_v,50_pairwise_v,80_pairwise_v,20_pairwise_acc,50_pairwise_acc,80_pairwise_acc,3ots_m_v,ete_larg_deviation,20_ete_line,50_ete_line,80_ete_line,ave_direction,length_trajectory,ratio_ete,ave_v,5points_m_acc,m_stroke_press,m_stroke_area_cover
False,20848,20848,20848,20848,20848,20848,20848,20848,20848,20848,20848,20848,20848,20848,20848,20848,20848,20848,20848,20848,20848,20848,20848,20848,20848,20848,20848,20848,20848,20848


# frank_feature_calc.csv

In [21]:
# 欠損値のカウント: Trueは欠損値
df_ffc2.isnull().apply(lambda col: col.value_counts(), axis=0).fillna(0).astype(np.int)

,user,doc,stroke_inter,stroke_duration,start_x,start_y,stop_x,stop_y,direct_ete_distance,mean_result_leng,flag,direct_ete_line,phone,20_pairwise_v,50_pairwise_v,80_pairwise_v,20_pairwise_acc,50_pairwise_acc,80_pairwise_acc,3ots_m_v,ete_larg_deviation,20_ete_line,50_ete_line,80_ete_line,ave_direction,length_trajectory,ratio_ete,ave_v,5points_m_acc,m_stroke_press,m_stroke_area_cover,finger_orien,cd_finger_orien,phone_orien
False,21173,21173,21133,21173,21173,21173,21173,21173,21173,21165,21173,21173,21173,21163,21163,21162,20907,20904,20907,20908,20493,20493,20493,20493,21165,21173,20903,21165,20916,21173,21173,21173,21173,21173
True,1,1,41,1,1,1,1,1,1,9,1,1,1,11,11,12,267,270,267,266,681,681,681,681,9,1,271,9,258,1,1,1,1,1


In [16]:
df_ffc2_drop = df_ffc2.drop({'phone', 'finger_orien', 'cd_finger_orien', 'phone_orien'}, axis=1)
df_ffc2_drop.head()

,user,doc,stroke_inter,stroke_duration,start_x,start_y,stop_x,stop_y,direct_ete_distance,mean_result_leng,flag,direct_ete_line,20_pairwise_v,50_pairwise_v,80_pairwise_v,20_pairwise_acc,50_pairwise_acc,80_pairwise_acc,3ots_m_v,ete_larg_deviation,20_ete_line,50_ete_line,80_ete_line,ave_direction,length_trajectory,ratio_ete,ave_v,5points_m_acc,m_stroke_press,m_stroke_area_cover
0,37.0,4.0,0.341,0.256,27.415873,-27.113492,12.397619,-32.959524,16.115956,0.357365,4.0,-2.770378,2.089222,4.178445,1512.514013,-432.239456,-432.239456,-432.239456,1512.514013,9.172222,0.403175,4.989286,9.172222,-2.581235,16.132491,0.998975,63.017533,-432.239456,0.32,0.044444
1,37.0,4.0,0.913,0.086,21.771429,-30.036508,9.877778,-33.866667,12.495161,0.265040,4.0,-2.830046,0.000000,156.788711,502.370053,-21322.709457,5117.745334,88903.936269,394.015610,8.063492,0.000000,6.350000,8.063492,-2.199436,12.596318,0.991969,146.468877,5117.745334,0.60,0.088889
2,37.0,4.0,0.733,0.111,10.885714,-34.169048,26.911905,-34.673016,16.034113,0.991178,2.0,-0.031436,32.122213,143.853750,263.584103,-38945.522228,-3563.474273,18510.405771,188.821551,15.522222,-15.522222,-4.636508,-1.209524,0.086288,16.202833,0.989587,145.971705,-3563.474273,0.48,0.044444
3,37.0,4.0,0.448,0.094,19.150794,-34.572222,12.700000,-36.789683,6.821281,0.596472,4.0,-2.810497,17.359942,76.637825,230.616519,-9135.221517,-4604.073096,17834.521764,95.856637,4.233333,1.007937,3.225397,4.233333,-2.723130,6.967444,0.979022,74.121670,-4604.073096,0.40,0.044444
4,37.0,4.0,0.519,0.056,27.718254,-32.757937,14.615079,-37.293651,13.865998,0.678191,4.0,-2.808348,106.887510,275.572786,767.677216,-126175.290213,-21377.012627,217042.986149,244.681884,8.567460,2.066270,6.249206,8.013095,-2.717012,13.870901,0.999646,247.694691,-21377.012627,0.44,0.044444


In [22]:
# 欠損値のカウント: Trueは欠損値
df_ffc2_drop.isnull().apply(lambda col: col.value_counts(), axis=0).fillna(0).astype(np.int)

,user,doc,stroke_inter,stroke_duration,start_x,start_y,stop_x,stop_y,direct_ete_distance,mean_result_leng,flag,direct_ete_line,20_pairwise_v,50_pairwise_v,80_pairwise_v,20_pairwise_acc,50_pairwise_acc,80_pairwise_acc,3ots_m_v,ete_larg_deviation,20_ete_line,50_ete_line,80_ete_line,ave_direction,length_trajectory,ratio_ete,ave_v,5points_m_acc,m_stroke_press,m_stroke_area_cover
False,21173,21173,21133,21173,21173,21173,21173,21173,21173,21165,21173,21173,21163,21163,21162,20907,20904,20907,20908,20493,20493,20493,20493,21165,21173,20903,21165,20916,21173,21173
True,1,1,41,1,1,1,1,1,1,9,1,1,11,11,12,267,270,267,266,681,681,681,681,9,1,271,9,258,1,1


In [17]:
df_ffc2_drop.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21174 entries, 0 to 21173
Data columns (total 30 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   user                 21173 non-null  float64
 1   doc                  21173 non-null  float64
 2   stroke_inter         21133 non-null  float64
 3   stroke_duration      21173 non-null  float64
 4   start_x              21173 non-null  float64
 5   start_y              21173 non-null  float64
 6   stop_x               21173 non-null  float64
 7   stop_y               21173 non-null  float64
 8   direct_ete_distance  21173 non-null  float64
 9   mean_result_leng     21165 non-null  float64
 10  flag                 21173 non-null  float64
 11  direct_ete_line      21173 non-null  float64
 12  20_pairwise_v        21163 non-null  float64
 13  50_pairwise_v        21163 non-null  float64
 14  80_pairwise_v        21162 non-null  float64
 15  20_pairwise_acc      20907 non-null 

In [23]:
# 欠損値削除: how='any'で欠損値が1つでも含まれる行(axis=0)を削除
df_ffc2_drop_f = df_ffc2_drop.dropna(axis=0, how='any')
df_ffc2_drop_f.head()

,user,doc,stroke_inter,stroke_duration,start_x,start_y,stop_x,stop_y,direct_ete_distance,mean_result_leng,flag,direct_ete_line,20_pairwise_v,50_pairwise_v,80_pairwise_v,20_pairwise_acc,50_pairwise_acc,80_pairwise_acc,3ots_m_v,ete_larg_deviation,20_ete_line,50_ete_line,80_ete_line,ave_direction,length_trajectory,ratio_ete,ave_v,5points_m_acc,m_stroke_press,m_stroke_area_cover
0,37.0,4.0,0.341,0.256,27.415873,-27.113492,12.397619,-32.959524,16.115956,0.357365,4.0,-2.770378,2.089222,4.178445,1512.514013,-432.239456,-432.239456,-432.239456,1512.514013,9.172222,0.403175,4.989286,9.172222,-2.581235,16.132491,0.998975,63.017533,-432.239456,0.32,0.044444
1,37.0,4.0,0.913,0.086,21.771429,-30.036508,9.877778,-33.866667,12.495161,0.265040,4.0,-2.830046,0.000000,156.788711,502.370053,-21322.709457,5117.745334,88903.936269,394.015610,8.063492,0.000000,6.350000,8.063492,-2.199436,12.596318,0.991969,146.468877,5117.745334,0.60,0.088889
2,37.0,4.0,0.733,0.111,10.885714,-34.169048,26.911905,-34.673016,16.034113,0.991178,2.0,-0.031436,32.122213,143.853750,263.584103,-38945.522228,-3563.474273,18510.405771,188.821551,15.522222,-15.522222,-4.636508,-1.209524,0.086288,16.202833,0.989587,145.971705,-3563.474273,0.48,0.044444
3,37.0,4.0,0.448,0.094,19.150794,-34.572222,12.700000,-36.789683,6.821281,0.596472,4.0,-2.810497,17.359942,76.637825,230.616519,-9135.221517,-4604.073096,17834.521764,95.856637,4.233333,1.007937,3.225397,4.233333,-2.723130,6.967444,0.979022,74.121670,-4604.073096,0.40,0.044444
4,37.0,4.0,0.519,0.056,27.718254,-32.757937,14.615079,-37.293651,13.865998,0.678191,4.0,-2.808348,106.887510,275.572786,767.677216,-126175.290213,-21377.012627,217042.986149,244.681884,8.567460,2.066270,6.249206,8.013095,-2.717012,13.870901,0.999646,247.694691,-21377.012627,0.44,0.044444


In [24]:
# 欠損値のカウント: Trueは欠損値
df_ffc2_drop_f.isnull().apply(lambda col: col.value_counts(), axis=0).fillna(0).astype(np.int)

,user,doc,stroke_inter,stroke_duration,start_x,start_y,stop_x,stop_y,direct_ete_distance,mean_result_leng,flag,direct_ete_line,20_pairwise_v,50_pairwise_v,80_pairwise_v,20_pairwise_acc,50_pairwise_acc,80_pairwise_acc,3ots_m_v,ete_larg_deviation,20_ete_line,50_ete_line,80_ete_line,ave_direction,length_trajectory,ratio_ete,ave_v,5points_m_acc,m_stroke_press,m_stroke_area_cover
False,20448,20448,20448,20448,20448,20448,20448,20448,20448,20448,20448,20448,20448,20448,20448,20448,20448,20448,20448,20448,20448,20448,20448,20448,20448,20448,20448,20448,20448,20448


書き換えた特徴量算出プログラムのほうがデータの欠損が多い